In [1]:
import logging

logging.basicConfig(format='\033[93m%(asctime)s %(levelname)s %(name)s:\033[0m %(message)s', level=logging.INFO, datefmt='%I:%M:%S')
import warnings
import pandas as pd

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.simplefilter(action="ignore", category=pd.errors.SettingWithCopyWarning)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load Data

In [3]:
from NNTrade.source.market.quotes import QuoteSource, date
from NNTrade.source.market.client import QuoteSourceClientFinamCSV, TimeFrame
from NNTrade.source.market.cache.filecache.file_cache_quote import FileCacheQuote
from NNTrade.common.candle_col_name import INDEX, OPEN, HIGH, LOW, CLOSE, VOLUME
from src.future_extrem_deviation import get_extrem_rel_of, ExtreamType, get_extrem_of, get_extrem_rel_matrix_of

In [4]:
qsc = QuoteSourceClientFinamCSV("./quote/")
qsc.add_file("EURUSD.Minute1.csv","EURUSD", TimeFrame.m1)
fcq = FileCacheQuote("./cache/quote")
qs = QuoteSource(fcq, qsc)

02:03:22 INFO QuoteSourceClientCSV: File ./quote/EURUSD.Minute1.csv to stock EURUSD timeframe Minute1 added to client


In [5]:
quote_h1_m1 = qs.get("EURUSD",TimeFrame.H, from_date=date(2010,1,1), untill_date=date(2011,1,1), step_timeframe=TimeFrame.m1)
quote_h1_m1

,aggregated_start_date_time,open,low,high,close,volume,isLast
start_date_time,,,,,,,
2010-01-01 00:00:00,2010-01-01 00:00:00,1.4331,1.4327,1.4337,1.4332,0.0,0
2010-01-01 00:01:00,2010-01-01 00:00:00,1.4331,1.4327,1.4337,1.4331,0.0,0
2010-01-01 00:02:00,2010-01-01 00:00:00,1.4331,1.4327,1.4337,1.4329,0.0,0
2010-01-01 00:03:00,2010-01-01 00:00:00,1.4331,1.4327,1.4337,1.4331,0.0,0
2010-01-01 00:04:00,2010-01-01 00:00:00,1.4331,1.4327,1.4337,1.4332,0.0,0
...,...,...,...,...,...,...,...
2010-09-30 23:56:00,2010-09-30 23:00:00,1.3654,1.3571,1.3654,1.3602,0.0,0
2010-09-30 23:57:00,2010-09-30 23:00:00,1.3654,1.3571,1.3654,1.3601,0.0,0
2010-09-30 23:58:00,2010-09-30 23:00:00,1.3654,1.3571,1.3654,1.3595,0.0,0


In [6]:
quote_h1 = quote_h1_m1[quote_h1_m1["isLast"] == 1]
quote_h1.set_index("aggregated_start_date_time",inplace=True)
quote_h1.index.name = INDEX
quote_h1 = quote_h1[[OPEN, HIGH, LOW,CLOSE, VOLUME]]
quote_h1

,open,high,low,close,volume
start_date_time,,,,,
2010-01-01 00:00:00,1.4331,1.4341,1.4319,1.4326,0.0
2010-01-01 01:00:00,1.4325,1.4331,1.4322,1.4324,0.0
2010-01-01 02:00:00,1.4323,1.4325,1.4311,1.4312,0.0
2010-01-01 03:00:00,1.4314,1.4329,1.4310,1.4322,0.0
2010-01-01 04:00:00,1.4341,1.4341,1.4320,1.4329,0.0
...,...,...,...,...,...
2010-09-30 20:00:00,1.3627,1.3683,1.3623,1.3676,0.0
2010-09-30 21:00:00,1.3676,1.3677,1.3616,1.3630,0.0
2010-09-30 22:00:00,1.3630,1.3660,1.3623,1.3653,0.0


# Get Extrem of X candle

In [7]:
ext_rel = get_extrem_rel_of(quote_h1, 3, ExtreamType.Low)
ext_rel

,Rel_of_Low,Idx_of_Low,Shift_to_Low
start_date_time,,,
2010-01-01 00:00:00,-0.001396,2010-01-01 02:00:00,2
2010-01-01 01:00:00,-0.001047,2010-01-01 03:00:00,2
2010-01-01 02:00:00,-0.000908,2010-01-01 03:00:00,1
2010-01-01 03:00:00,-0.000279,2010-01-01 03:00:00,0
2010-01-01 04:00:00,-0.001464,2010-01-01 04:00:00,0
...,...,...,...
2010-09-30 20:00:00,-0.000807,2010-09-30 21:00:00,1
2010-09-30 21:00:00,-0.007678,2010-09-30 23:00:00,2
2010-09-30 22:00:00,-0.004329,2010-09-30 23:00:00,1


In [13]:
matrix_df = get_extrem_rel_matrix_of(quote_h1, [4,8,12,16,40])
matrix_df

4                                                 
                    Rel_of_High         Idx_of_High Shift_to_High Rel_of_Low   
start_date_time                                                                
2010-01-01 00:00:00    0.000698 2010-01-01 00:00:00             0  -0.001465  \
2010-01-01 01:00:00    0.001117 2010-01-01 04:00:00             3  -0.001047   
2010-01-01 02:00:00    0.001257 2010-01-01 04:00:00             2  -0.000908   
2010-01-01 03:00:00    0.001886 2010-01-01 04:00:00             1  -0.000279   
2010-01-01 04:00:00    0.000000 2010-01-01 04:00:00             0  -0.001464   
...                         ...                 ...           ...        ...   
2010-09-30 20:00:00    0.004109 2010-09-30 20:00:00             0  -0.004109   
2010-09-30 21:00:00    0.000073 2010-09-30 21:00:00             0  -0.007678   
2010-09-30 22:00:00    0.002201 2010-09-30 22:00:00             0  -0.004329   
2010-09-30 23:00:00    0.000000 2010-09-30 23:00:00             0  -0.006079   
2010-10-01 00:00:00    0.000294 2010-10-01 00:00:00             0  -0.000294   

                                                              8    
                             Idx_of_Low Shift_to_Low Rel_of_High   
start_date_time                                                    
2010-01-01 00:00:00 2010-01-01 03:00:00            3    0.000698  \
2010-01-01 01:00:00 2010-01-01 03:00:00            2    0.001117   
2010-01-01 02:00:00 2010-01-01 03:00:00            1    0.001257   
2010-01-01 03:00:00 2010-01-01 03:00:00            0    0.001886   
2010-01-01 04:00:00 2010-01-01 04:00:00            0    0.000000   
...                                 ...          ...         ...   
2010-09-30 20:00:00 2010-09-30 23:00:00            3    0.004109   
2010-09-30 21:00:00 2010-09-30 23:00:00            2    0.000073   
2010-09-30 22:00:00 2010-09-30 23:00:00            1    0.002201   
2010-09-30 23:00:00 2010-09-30 23:00:00            0    0.000000   
2010-10-01 00:00:00 2010-10-01 00:00:00            0    0.000294   

                                                                  ...   
                            Idx_of_High Shift_to_High Rel_of_Low  ...   
start_date_time                                                   ...   
2010-01-01 00:00:00 2010-01-01 00:00:00             0  -0.001465  ...  \
2010-01-01 01:00:00 2010-01-01 04:00:00             3  -0.001047  ...   
2010-01-01 02:00:00 2010-01-01 04:00:00             2  -0.000908  ...   
2010-01-01 03:00:00 2010-01-01 04:00:00             1  -0.000279  ...   
2010-01-01 04:00:00 2010-01-01 04:00:00             0  -0.001674  ...   
...                                 ...           ...        ...  ...   
2010-09-30 20:00:00 2010-09-30 20:00:00             0  -0.004109  ...   
2010-09-30 21:00:00 2010-09-30 21:00:00             0  -0.007678  ...   
2010-09-30 22:00:00 2010-09-30 22:00:00             0  -0.004329  ...   
2010-09-30 23:00:00 2010-09-30 23:00:00             0  -0.006079  ...   
2010-10-01 00:00:00 2010-10-01 00:00:00             0  -0.000294  ...   

                               16                                               
                    Shift_to_High Rel_of_Low          Idx_of_Low Shift_to_Low   
start_date_time                                                                 
2010-01-01 00:00:00             0  -0.001465 2010-01-01 03:00:00            3  \
2010-01-01 01:00:00             3  -0.001047 2010-01-01 03:00:00            2   
2010-01-01 02:00:00             2  -0.000908 2010-01-01 03:00:00            1   
2010-01-01 03:00:00             1  -0.000279 2010-01-01 03:00:00            0   
2010-01-01 04:00:00             0  -0.001674 2010-01-01 10:00:00            6   
...                           ...        ...                 ...          ...   
2010-09-30 20:00:00             0  -0.004109 2010-09-30 23:00:00            3   
2010-09-30 21:00:00             0  -0.007678 2010-09-30 23:00:00            2   
2010-09-30 22:00:00             0  -0.004

In [14]:
matrix_df[[c for c in matrix_df.columns if "Rel_of" in c[1]]].describe()

4                         8                         12   
       Rel_of_High   Rel_of_Low  Rel_of_High   Rel_of_Low  Rel_of_High   
count  4860.000000  4860.000000  4860.000000  4860.000000  4860.000000  \
mean      0.002164    -0.002282     0.003084    -0.003227     0.003803   
std       0.002135     0.002030     0.002942     0.002720     0.003474   
min       0.000000    -0.016419     0.000000    -0.021408     0.000000   
25%       0.000737    -0.003025     0.001037    -0.004365     0.001257   
50%       0.001505    -0.001716     0.002126    -0.002497     0.002724   
75%       0.002875    -0.000877     0.004208    -0.001298     0.005314   
max       0.021323     0.000000     0.021406     0.000000     0.026653   

                             16                        40               
        Rel_of_Low  Rel_of_High   Rel_of_Low  Rel_of_High   Rel_of_Low  
count  4860.000000  4860.000000  4860.000000  4860.000000  4860.000000  
mean     -0.003973     0.004431    -0.004629     0.007041    -0.007556  
std       0.003255     0.003918     0.003727     0.005984     0.006199  
min      -0.023249     0.000000    -0.025378     0.000000    -0.036404  
25%      -0.005402     0.001492    -0.006348     0.002471    -0.010286  
50%      -0.003133     0.003363    -0.003729     0.005850    -0.006016  
75%      -0.001611     0.006279    -0.001861     0.009835    -0.002866  
max       0.000000     0.027616     0.000000     0.038218     0.000000